# Data Processes
state_dict = {"sleep":0, "preseizure": 1}

In [30]:
import numpy as np
import os 
import pickle
import sys
sys.path.append("../")
from util.util_file import time_add

## 1.构建滤波数据和原始数据的映射 

In [59]:
# filter
filter_pre_1_path_dir = "../visualization_feature/raw_data_time_sequentially/preseizure/BDP/filter/pre_1"
filter_pre_2_path_dir = "../visualization_feature/raw_data_time_sequentially/preseizure/BDP/filter/pre_2"
filter_sleep_path_dir = "..//visualization_feature/raw_data_time_sequentially/sleep/BDP/filter"

# non-filter
non_filter_pre_1_path_dir = "../visualization_feature/raw_data_time_sequentially/preseizure/BDP/no_filter/pre_1"
non_filter_pre_2_path_dir = "../visualization_feature/raw_data_time_sequentially/preseizure/BDP/no_filter/pre_2"
non_filter_sleep_path_dir = "..//visualization_feature/raw_data_time_sequentially/sleep/BDP/no_filter"

生成滤波和非滤波切片的映射

In [60]:
fi_pre_1_names = os.listdir(filter_pre_1_path_dir)
fi_pre_2_names = os.listdir(filter_pre_2_path_dir)
fi_sleep_names = os.listdir(filter_sleep_path_dir)
# sorted by time 
fi_pre_1_names.sort()
fi_pre_2_names.sort()
fi_sleep_names.sort()
print("fi_pre_1:{}, fi_pre_2:{}, fi_sleep:{}".format(len(fi_pre_1_names), len(fi_pre_2_names), len(fi_sleep_names)))

nfi_pre_1_names = os.listdir(non_filter_pre_1_path_dir)
nfi_pre_2_names = os.listdir(non_filter_pre_2_path_dir)
nfi_sleep_names = os.listdir(non_filter_sleep_path_dir)
nfi_pre_1_names.sort()
nfi_pre_2_names.sort()
nfi_sleep_names.sort()
print("nfi_pre_1:{}, nfi_pre_2:{}, nfi_sleep:{}".format(len(nfi_pre_1_names), len(nfi_pre_2_names), len(nfi_sleep_names)))


fi_nfi_dict = {}
label = 1
for m, n in zip(fi_pre_1_names+fi_pre_2_names, nfi_pre_1_names+nfi_pre_2_names):
    fi_nfi_dict[m] = {"nfi_id":n, "label": label}
label = 0
for m ,n in zip(fi_sleep_names, nfi_sleep_names):
    fi_nfi_dict[m] = {"nfi_id":n, "label": label}

print(len(fi_nfi_dict))


fi_pre_1:309, fi_pre_2:1783, fi_sleep:7199
nfi_pre_1:309, nfi_pre_2:1783, nfi_sleep:7199
9291


In [61]:
print(fi_pre_1_names)

['e076f2d4-2552-11ea-9699-e0d55e6ff654-0.npy', 'e076f2d5-2552-11ea-9699-e0d55e6ff654-0.npy', 'e076f2d6-2552-11ea-9699-e0d55e6ff654-0.npy', 'e076f2d7-2552-11ea-9699-e0d55e6ff654-0.npy', 'e076f2d8-2552-11ea-9699-e0d55e6ff654-0.npy', 'e076f2d9-2552-11ea-9699-e0d55e6ff654-0.npy', 'e076f2da-2552-11ea-9699-e0d55e6ff654-0.npy', 'e076f2db-2552-11ea-9699-e0d55e6ff654-0.npy', 'e076f2dc-2552-11ea-9699-e0d55e6ff654-0.npy', 'e076f2dd-2552-11ea-9699-e0d55e6ff654-0.npy', 'e076f2de-2552-11ea-9699-e0d55e6ff654-0.npy', 'e076f2df-2552-11ea-9699-e0d55e6ff654-0.npy', 'e076f2e0-2552-11ea-9699-e0d55e6ff654-0.npy', 'e076f2e1-2552-11ea-9699-e0d55e6ff654-0.npy', 'e076f2e2-2552-11ea-9699-e0d55e6ff654-0.npy', 'e076f2e3-2552-11ea-9699-e0d55e6ff654-0.npy', 'e076f2e4-2552-11ea-9699-e0d55e6ff654-0.npy', 'e076f2e5-2552-11ea-9699-e0d55e6ff654-0.npy', 'e076f2e6-2552-11ea-9699-e0d55e6ff654-0.npy', 'e076f2e7-2552-11ea-9699-e0d55e6ff654-0.npy', 'e076f2e8-2552-11ea-9699-e0d55e6ff654-0.npy', 'e076f2e9-2552-11ea-9699-e0d55e6f

将映射表进行保存

保存格式：{fi_id:{"nfi_id":nfi_id, "label":0/1 }}

fi_id: 经过滤波切片的id，  nfi_if: 没有经过滤波的切片id

In [62]:
sava_file = "./data/fi_nfi_id.pkl"
with open(sava_file, 'wb') as f:
    pickle.dump(fi_nfi_dict, f)
    print("映射表保存成功！")

映射表保存成功！


## 2.特征重新映射,需要将特征和id序列号进行映射

In [63]:
feature_pre_1_path = "./data/pre_1/BDP-feature.npy"
feature_pre_2_path = "./data/pre_2/BDP-feature.npy"
feature_sleep_path = "./data/sleep/BDP-feature.npy"

feature_pre_1_data = np.load(feature_pre_1_path)
feature_pre_2_data = np.load(feature_pre_2_path)
feature_sleep_data = np.load(feature_sleep_path)

print(feature_pre_1_data.shape, feature_pre_2_data.shape, feature_sleep_data.shape)
feature_id_data_dict = {}
concatenate_feature = np.concatenate((feature_pre_1_data, feature_pre_2_data, feature_sleep_data), axis=0)
print(concatenate_feature.shape)
for id, data in zip(fi_pre_1_names+fi_pre_2_names+fi_sleep_names, concatenate_feature):
    feature_id_data_dict[id] = data
print(len(feature_id_data_dict))

(309, 50) (1783, 50) (7199, 50)
(9291, 50)
9291


In [64]:
save_path = "./data/feature_id_data_dict.pkl"
with open(save_path, 'wb') as f:
    pickle.dump(feature_id_data_dict, f)
    print("特征id序列化完成！")

特征id序列化完成！


## 3.选出模型判定正确的特征


In [65]:
path_val_prediction = "./data/BDP_val_prediction.pkl"
val_prediction_dict = np.load(path_val_prediction, allow_pickle=True)

In [66]:
a = val_prediction_dict["e076f3cc-2552-11ea-9699-e0d55e6ff654-0.npy"]["ground truth"]
b = val_prediction_dict["e076f3cc-2552-11ea-9699-e0d55e6ff654-0.npy"]["prediction"]
print(a == b)

True


In [68]:
feature_true_id_prediction = {}
for id, data in feature_id_data_dict.items():
    if id in val_prediction_dict.keys():
        if val_prediction_dict[id]["ground truth"] == val_prediction_dict[id]["prediction"]:
            feature_true_id_prediction[id] = data

print(len(feature_true_id_prediction))
    

4286


## 4. 每个特征和时间的映射关系

In [32]:
feature_id_time_dict = {}
pre_1_time = (23, 1, 52)
pre_2_time = (3, 1, 52)
sleep_time = (23, 1, 52)
_time = pre_1_time
for id in fi_pre_1_names:
    feature_id_time_dict[id] = _time
    _time = time_add(_time[0], _time[1], _time[2], 2)

_time = pre_2_time
for id in fi_pre_2_names:
    feature_id_time_dict[id] = _time
    _time = time_add(_time[0], _time[1], _time[2], 2)

_time = sleep_time
for id in fi_sleep_names:
    feature_id_time_dict[id] = _time
    _time = time_add(_time[0], _time[1], _time[2], 2)

save_path = "./data/feature_id_time_dict.pkl"
print(len(feature_id_time_dict))
with open(save_path, 'wb') as f:
    pickle.dump(feature_id_time_dict, f)
    print("特征时间映射完成！")
    

9291
特征时间映射完成！


(1, 0, 0)
